# TABLE OF CONTENTS

* [1. INTRODUCTION](#section-one)
* [2. SETUP](#section-two)
    - [2.1 Install and Load Libraries](#subsection-two-one)
    - [2.2 Define Parameters](#subsection-two-two)
    - [2.3 Import Data](#subsection-two-three)
    - [2.4 Wrangle Data](#subsection-two-four)
* [3. STORY](#section-three)
    - [3.1 Question 1: How has Covid19 spread around the world?](#subsection-three-one)
    - [3.2 Question 2: Are country-level Covid19 pandemic curves different?](#subsection-three-two)
    - [3.3 Question 3: Are the Reported Confirmed Cases and Deaths Reliable? Case Positivity and Case Fatality Rate?](#subsection-three-three)
    - [3.4 Question 4: How do we estimate the True Infection Rate?](#subsection-three-four)
    
* [4. CONCLUSION](#section-four)
* [5. REFERENCES](#section-five)

<a id='section-one'></a>
# 1. INTRODUCTION

In this notebook, we explore how the Covid19 pandemic has evolved geographically over time around the world and whether the Covid19 pandemic curves are different across the world. Then, we take a deep delve into the reported confirmed cases and case fatality rates. What may drive the two measures? Are the case fatality rates reliable measures of true fatality rates of the Covid19? What is the problem with the case fatality rate? How can we estimate the true infection rate?

<a id='subsection-two-one'></a>
## 2.1 Install and Load Libraries

<a id='section-two'></a>
# 2. SETUP 

FOR KAGGLE (IGNORE FOR JUPYTERLAB)
1. Settings -> Environment -> 'Always use latest environment'
2. Settings -> Internet -> 'On'

In [ ]:
# math operations
from numpy import inf

# time operations
from datetime import timedelta

# for numerical analyiss
import numpy as np

# to store and process data in dataframe
import pandas as pd

# basic visualization package
import matplotlib.pyplot as plt

# advanced ploting
import seaborn as sns

# interactive visualization
import plotly.express as px
import plotly.graph_objs as go

# import plotly.figure_factory as ff
# from plotly.subplots import make_subplots

# for offline ploting
from plotly.offline import plot, iplot, init_notebook_mode

init_notebook_mode(connected=True)

# hide warnings
import warnings

warnings.filterwarnings("ignore")


<a id='subsection-two-two'></a>
## 2.2 Define Parameters

In [ ]:
# color pallette
cnf, dth, rec, act = "#393e46", "#ff2e63", "#21bf73", "#fe9801"

<a id='subsection-two-three'></a>
## 2.3 Import Data
Please add Covid 19 data from https://www.kaggle.com/imdevskp/corona-virus-report  
and Worldometer snapshot data from https://www.kaggle.com/selfishgene/covid19-worldometer-snapshots-since-april-18


In [ ]:
# A. Kaggle: list files
!ls ../input/corona-virus-report

# B. Local: download all files into the folder where the notebook is
# local_path = 'C:/jupyter_workspace/Fintech/Codes/Lab 4 Deep Diving into COVID19/data/'

## 2.3a Import Data Via CLI

In [ ]:
### API download requires kaggle API key in .kaggle file
### The API downloads a zipped file, manual extraction still required. 
### !kaggle datasets download -d imdevskp/corona-virus-report --force

<a id='subsection-two-four'></a>
## 2.4 Wrangle Data

In [ ]:
# Full data
# =========
# on Kaggle
full_table = pd.read_csv('../input/corona-virus-report/covid_19_clean_complete.csv')
# full_table = pd.read_csv("data/covid_19_clean_complete.csv")

# Deep dive into the DataFrame
# Examine DataFrame (object type, shape, columns, dtypes)
full_table.info()

# type(full_table)
# full_table.shape
# full_table.columns
# full_table.dtypes
# full_table.head(20)

In [ ]:
# Country wise
# ============

country_wise = pd.read_csv('../input/corona-virus-report/country_wise_latest.csv')
# country_wise = pd.read_csv('data/country_wise_latest.csv')


# Replace missing values '' with NAN and then 0
country_wise = country_wise.replace('', np.nan).fillna(0)

# Deep dive into the DataFrame
country_wise.info()
country_wise.head(10)


In [ ]:
# Grouped by day, country
# =======================

full_grouped = pd.read_csv('../input/corona-virus-report/full_grouped.csv')
# full_grouped = pd.read_csv("data/full_grouped.csv")
full_grouped.info()
full_grouped.head(10)

# Convert Date from Dtype 'Object' (or String) to Dtype 'Datetime'
full_grouped["Date"] = pd.to_datetime(full_grouped["Date"])
full_grouped.info()

In [ ]:
# Day wise
# ========

day_wise = pd.read_csv('../input/corona-virus-report/day_wise.csv')
# day_wise = pd.read_csv("data/day_wise.csv")
day_wise["Date"] = pd.to_datetime(day_wise["Date"])
day_wise.info()
day_wise.head(10)

In [ ]:
# Worldometer data
# ================

worldometer_data = pd.read_csv('../input/corona-virus-report/worldometer_data.csv')
# worldometer_data = pd.read_csv('data/worldometer_data.csv')

# Replace missing values '' with NAN and then 0
# What are the alternatives? Drop or impute. Do they make sense in this context?
worldometer_data = worldometer_data.replace('', np.nan).fillna(0)
worldometer_data['Case Positivity'] = round(worldometer_data['TotalCases']/worldometer_data['TotalTests'],2)
worldometer_data['Case Fatality'] = round(worldometer_data['TotalDeaths']/worldometer_data['TotalCases'],2)

# Case Positivity is infinity when there is zero TotalTests due to division by zero
worldometer_data[worldometer_data['Case Positivity'] == inf] = 0

# Qcut is quantile cut. Here we specify three equally sized bins and label them low, medium, and high, respectively.
worldometer_data ['Case Positivity Bin']= pd.qcut(worldometer_data['Case Positivity'], q=3, labels=['low', 'medium', 'high'])

# Population Structure
worldometer_pop_struc = pd.read_csv('../input/covid19-worldometer-snapshots-since-april-18/population_structure_by_age_per_contry.csv')
# worldometer_pop_struc = pd.read_csv('data/population_structure_by_age_per_contry.csv')

# Replace missing values with zeros
worldometer_pop_struc = worldometer_pop_struc.fillna(0)
#worldometer_pop_struc.info()

# Merge worldometer_data with worldometer_pop_struc
# Inner means keep only common key values in both datasets
worldometer_data = worldometer_data.merge(worldometer_pop_struc,how='inner',left_on='Country/Region', right_on='Country')

# Keep observations where column 'Country/Region' is not 0
worldometer_data = worldometer_data[worldometer_data['Country/Region'] != 0]

# Inspect worldometer_data's metadata
worldometer_data.info()

# Inspect Data
# worldometer_data.info()
# worldometer_data.tail(20)
# worldometer_data['Case Positivity'].describe()


Details on qcut [HERE](https://www.geeksforgeeks.org/how-to-use-pandas-cut-and-qcut/).   
Details on merge [HERE](https://towardsdatascience.com/why-and-how-to-use-merge-with-pandas-in-python-548600f7e738#:~:text=3.-,INNER%20Merge,in%20the%20merged%20data%20%E2%80%94%20inner_merge.).   
Details on the difference between merge, join, and concat [HERE](https://realpython.com/pandas-merge-join-and-concat/).

<a id='section-three'></a>
# 3. STORY

<a id='subsection-three-one'></a>
## 3.1 Question 1: How has Covid19 spread around the world?

In [ ]:
# The three arguments of the function are:
# 1. input dataframe
# 2. variable of interest
# 3. color palette
def plot_map(df, col, pal):
    """
    Function plots on a world map the total number of cases by country names 
    
    Parameters:
        df (pandas.DataFrame): Dataframe containing the number of cases in each country 
        col (str): The column name of interest 
        pal (str): color palette to use
    Returns:
        null
    """
    df = df[df[col] > 0]

    # choropleth draws maps based on shape files of different countries
    fig = px.choropleth(
        df,
        locations="Country/Region",
        locationmode="country names",
        color=col,
        hover_name="Country/Region",
        title=col,
        hover_data=[col],
        color_continuous_scale=pal,
    )
    # 1:50m resolution
    fig.update_geos(resolution=50)
    fig.show()

More details Plotly's choropleth [HERE](https://plotly.com/python/map-configuration/).

In [ ]:
# Inspect dataframe worldometer_data
worldometer_data.info()

In [ ]:
# Plot 'TotalCases' on world map using worldometer_data as the input dataframe with color palette 'matter'
plot_map(worldometer_data, "TotalCases", "matter")

More details on Plotly map configuration [HERE](https://plotly.com/python/map-configuration/).

In [ ]:
# What do you read this?
plot_map(worldometer_data, "Tot Cases/1M pop", "matter")

In [ ]:
# Plot log(Confirmed Cases) for each Country/Region Over time
fig = px.choropleth(
    full_grouped,
    locations="Country/Region",
    color=np.log(full_grouped["Confirmed"]),
    locationmode="country names",
    hover_name="Country/Region",
    animation_frame=full_grouped["Date"].dt.strftime("%Y-%m-%d"),
    title="Cases over time",
    color_continuous_scale=px.colors.sequential.matter,
)
fig.update(layout_coloraxis_showscale=False)
fig.update_geos(resolution=50)
fig.show()

<a id='subsection-three-two'></a>
## 3.2 Question 2: Do country-level Covid19 pandemic curves differ?

### Recap: Covid19 progression over time

In [ ]:
temp = full_grouped.groupby("Date")["Recovered", "Deaths", "Active"].sum().reset_index()
temp = temp.melt(
    id_vars="Date",
    value_vars=["Recovered", "Deaths", "Active"],
    var_name="Case",
    value_name="Count",
)
temp.head()

# Plot a stack area graph with the three types of cases (i.e., recovered, deaths, and active)
fig = px.area(
    temp,
    x="Date",
    y="Count",
    color="Case",
    height=600,
    width=700,
    title="Cases over time",
    color_discrete_sequence=[rec, dth, act],
)
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
def gt_n(n):
    # Identify countries with confirmed cases greater than N
    # Then among these countries choose the unique set of countries
    countries = full_grouped[full_grouped["Confirmed"] > n]["Country/Region"].unique()

    # Filter countries that are in the unique set of countries with confirmed cases greater than N
    temp = full_table[full_table["Country/Region"].isin(countries)]

    # Aggregate (i.e., sum up) confirmed cases by Country/Region and Date
    # Reset the index (it is no longer in running order)
    temp = temp.groupby(["Country/Region", "Date"])["Confirmed"].sum().reset_index()

    # Filter observations with confirmed cases more than N
    temp = temp[temp["Confirmed"] > n]
    # print(temp.head())

    # Identify the start date when confirmed cases exceed N for each country
    min_date = temp.groupby("Country/Region")["Date"].min().reset_index()

    # Name the columns in the dataframe min_date
    min_date.columns = ["Country/Region", "Min Date"]
    # print(min_date.head())

    # Merge dataframe temp with dataframe min_date by 'Country/Region'
    from_nth_case = pd.merge(temp, min_date, on="Country/Region")

    # Convert data type to datetime object
    from_nth_case["Date"] = pd.to_datetime(from_nth_case["Date"])
    from_nth_case["Min Date"] = pd.to_datetime(from_nth_case["Min Date"])

    # Create a variable that counts the number of days relative to the day when confirmed cases exceed N
    from_nth_case["N days"] = (
        from_nth_case["Date"] - from_nth_case["Min Date"]
    ).dt.days
    # print(from_nth_case.head())

    # Plot a line graph from dataframe from_nth_case with column 'N days' and 'Confirmed' mapped to x-axis and y-axis, respectively.
    # Distinguish each country by color (system-determined color)
    # str converts n integer into string and ''N days from '+ str(n) +' case'' is the title
    fig = px.line(
        from_nth_case,
        x="N days",
        y="Confirmed",
        color="Country/Region",
        title="N days from " + str(n) + " case",
        height=600,
    )
    fig.show()

## EDITED

In [ ]:
def graph_cty_exceeding_cases(n, full_grouped_df, full_table_df):
    """
    Function graphs the countries with more than n confirmed cases 
    
    Parameters:
        n (int): The threshold minimum number of cases 
        fully_grouped_df (pandas.DataFrame): the data source used. From Kaggle's full_grouped data set 
        full_data_df (pandas.DataFrame): the data source used. From Kaggle's covid_19_clean_complete data set 
    Returns:
        null
    """

    # Data Preprocessing
    full_grouped_df["Date"] = pd.to_datetime(full_grouped_df["Date"])

    temp = (
        full_grouped_df.groupby("Date")["Recovered", "Deaths", "Active"]
        .sum()
        .reset_index()
    )
    temp = temp.melt(
        id_vars="Date",
        value_vars=["Recovered", "Deaths", "Active"],
        var_name="Case",
        value_name="Count",
    )

    # Identify countries with confirmed cases greater than N
    # Then among these countries choose the unique set of countries
    countries = full_grouped_df[full_grouped_df["Confirmed"] > n][
        "Country/Region"
    ].unique()

    # Filter countries that are in the unique set of countries with confirmed cases greater than N
    temp = full_table_df[full_table_df["Country/Region"].isin(countries)]

    # Aggregate (i.e., sum up) confirmed cases by Country/Region and Date
    # Reset the index (it is no longer in running order)
    temp = temp.groupby(["Country/Region", "Date"])["Confirmed"].sum().reset_index()

    # Filter observations with confirmed cases more than N
    temp = temp[temp["Confirmed"] > n]
    # print(temp.head())

    # Identify the start date when confirmed cases exceed N for each country
    min_date = temp.groupby("Country/Region")["Date"].min().reset_index()

    # Name the columns in the dataframe min_date
    min_date.columns = ["Country/Region", "Min Date"]
    # print(min_date.head())

    # Merge dataframe temp with dataframe min_date by 'Country/Region'
    from_nth_case = pd.merge(temp, min_date, on="Country/Region")

    # Convert data type to datetime object
    from_nth_case["Date"] = pd.to_datetime(from_nth_case["Date"])
    from_nth_case["Min Date"] = pd.to_datetime(from_nth_case["Min Date"])

    # Create a variable that counts the number of days relative to the day when confirmed cases exceed N
    from_nth_case["N days"] = (
        from_nth_case["Date"] - from_nth_case["Min Date"]
    ).dt.days
    # print(from_nth_case.head())

    # Plot a line graph from dataframe from_nth_case with column 'N days' and 'Confirmed' mapped to x-axis and y-axis, respectively.
    # Distinguish each country by color (system-determined color)
    # str converts n integer into string and ''N days from '+ str(n) +' case'' is the title
    fig = px.line(
        from_nth_case,
        x="N days",
        y="Confirmed",
        color="Country/Region",
        title="N days from " + str(n) + " case",
        height=600,
    )
    fig.show()

In [ ]:
# Call function gt_n with argument 100000 (i.e., 100000 confirmed cases)
graph_cty_exceeding_cases(100000, full_grouped, full_table)

The pandemic curves are different across countries. While some countries manage to tame the pandemic, others continue to experience outbreak. What do you think contribute to these differences?

<a id='subsection-three-three'></a>
## 3.3 Question 3: Are the Reported Confirmed Cases and Deaths Reliable? Case Positivity and Case Fatality Rate.

## EDITED

In [ ]:
# Draw horizontal bar plot with three arguments
# 1. variable of interest
# 2. top n countries
# 3. minimum population size (default value is 1000000)
def plot_hbar_wm(
    worldometer_data_df, col, n, min_pop=1000000,
):
    """
    Function plots a horizontal barchart for countries on a variable of interest, 
    with a pre-specified minimum population size and top number of countries to show
    
    Parameters:
        worldometer_data_df (pandas.DataFrame): the dataframe used as a data source. From Kaggle's worldometer_data data set
        col (str): Variable of interest (Column Label)
        n (int): top n number of countries to graph 
        min_pop (int): the threshold minimum population size 
    Returns:
        null 
    """
    df = worldometer_data_df[worldometer_data_df["Population"] > min_pop]
    df = df.sort_values(col, ascending=True).tail(n)
    df.info()
    fig = px.bar(
        df,
        x=col,
        y="Country/Region",
        color="WHO Region",
        text=col,
        orientation="h",
        width=700,
        color_discrete_sequence=px.colors.qualitative.Dark2,
    )
    fig.update_layout(
        title=col + " (Only countries with Population > " + str(min_pop),
        xaxis_title="",
        yaxis_title="",
        yaxis_categoryorder="total ascending",
        uniformtext_minsize=8,
        uniformtext_mode="hide",
    )
    fig.show()


# Draw histogram with two arguments
# 1. variable of interest
# 2. the number of bins
def plot_histogram_wm(col, bins, worldometer_data_df):
    """
    Function plots a histogram using worldometer data, with a pre-specified number of bins 
    
    Parameters:
        col (str): The variable of interest (Column Label)
        bins (int): The number of bins to be displayed
        worldometer_data_df (pandas.DataFrame): The datasource to be used. From Kaggle's worldometer_data set
    Returns:
        null
    """
    fig = px.histogram(worldometer_data_df[col], x=col, nbins=bins)
    fig.show()

In [ ]:
# Draw bar chart for case fatality of the top 15 countries with the highest case fatality rate (with the minimum population of 1 million)
plot_hbar_wm(worldometer_data, "Case Fatality", 15, 1000000)

Yemen has the highest case fatality rate. What is going on? The next five countries are European countries. Is the same factor driving Yemen's high fatality rate at work here?

In [ ]:
# Draw the histogram for case fatality rate (50 bins)
plot_histogram_wm("Case Fatality", 50, worldometer_data)

In [ ]:
# Draw a joint plot to diagnose the relationship between Tests/1M population vs Total Cases/1M population
# Draw a regression line on the scatter plot
sns.jointplot(x = 'Tests/1M pop', y = 'Tot Cases/1M pop', data = worldometer_data, kind='reg')

In [ ]:
# Show the descriptive statistics for case positivity bin (categorical variable)
worldometer_data.groupby('Case Positivity Bin')['Case Positivity Bin'].describe()

In [ ]:
# Draw a Violin plot to diagnose the relationship betwen case positivity and case fatality rates
fig = go.Figure()

# Create a list of case positivity bin categories
bins = ['low', 'medium', 'high']

# Loop through each case positivity bin
for bin in bins:
    
    # worldometer_data['Case Positivity Bin'][worldometer_data['Case Positivity Bin'] == bin] means take the column 'Case Positivity Bin' and
    # filter the column, such that Case Positivity Bin equals 'low', 'medium', or 'high'
    fig.add_trace(go.Violin(x=worldometer_data['Case Positivity Bin'][worldometer_data['Case Positivity Bin'] == bin],
                            y=worldometer_data['Case Fatality'][worldometer_data['Case Positivity Bin'] == bin],
                            name=bin,
                            box_visible=True,
                            meanline_visible=True))
    
fig.update_layout(title='Case Fatality by Case Positivity Bins', 
                  yaxis_title='Case Fatality', xaxis_title='Case Positivity Bins', 
                  uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

The higher the case positivity bin the higher the case fatality rate.

In [ ]:
# Draw a joint plot to diagnose the relationship between fraction of population aged 65+ and case fatality rate
sns.jointplot(x = 'Fraction age 65+ years', y = 'Case Fatality', data = worldometer_data, kind='reg')

Case fatality rate is a function of the fraction of age 65+ years in the population.

<a id='subsection-three-four'></a>
## 3.4 Question 4: How do we estimate the True Fatality Rate?

In [ ]:
# Show the summary statistics of column case positivity
worldometer_data['Case Positivity'].describe()

# Filter countries with Case Positivity less than 1% (i.e., 1 confirmed case out of 100 tests)
# These are countries that go for rigorous testing regime
benchmark_countries = worldometer_data[worldometer_data['Case Positivity']<=0.01]
#benchmark_countries.info()
#benchmark_countries.head(20)

In [ ]:
# Assume that the number of confirmed cases are close to the true infections rates for countries with gold standard testing regimes 
# Thus, their case fatality rates are closer to the true infection fatality rates
infection_fatality_rate = benchmark_countries['TotalDeaths'].sum() / benchmark_countries['TotalCases'].sum()

# Calculate the fraction of total Covid19 deaths for the population aged 65+ among the benchmark countries
benchmark_death_65y_pct = sum(benchmark_countries['TotalDeaths'] * benchmark_countries['Fraction age 65+ years']) / sum(benchmark_countries['TotalDeaths'])

print(infection_fatality_rate)
print(benchmark_death_65y_pct)

print('Estimated Infection Fatality Rate for a benchmark country with %.1f%s of population older than 65 years old \
is %.2f%s' %(100 * benchmark_death_65y_pct,'%',100 * infection_fatality_rate,'%'))

Case Fatality Rate is positively correlated with the fraction of 65+ in the population. Here we calibrate each country's Infections Fatality Rate following the estimated fraction of total Covid19 deaths for the population aged 65+ (i.e., we expect the Infections Fatality Rate of a country with 20% estimated total Covid19 deaths among 65+ year old population to be twice higher than another country with 10%).

In [ ]:
# Estimate Infection Fatality Ratio using the estimated fraction of total Covid19 deaths for the population aged 65+
worldometer_data['Estimated Infection Fatality Ratio'] \
    = ((worldometer_data['TotalDeaths'] * worldometer_data['Fraction age 65+ years']
        /worldometer_data['TotalDeaths']) / benchmark_death_65y_pct) * infection_fatality_rate

# Show descriptive statistics of the columns Estimated Infection Fatality Ratio and Case Fatality
worldometer_data['Estimated Infection Fatality Ratio'].describe()
worldometer_data['Case Fatality'].describe()

# Plot histogram of Estimated Infection Fatality Ratio and Case Fatality
px.histogram(worldometer_data, x='Estimated Infection Fatality Ratio', barmode='overlay')
px.histogram(worldometer_data, x='Case Fatality', barmode='overlay')

# Overlay both histograms for comparison
fig = go.Figure()

fig.add_trace(go.Histogram(x=worldometer_data['Estimated Infection Fatality Ratio'], 
    name = 'Estimated Infection Fatality Rate'
))

fig.add_trace(go.Histogram(x=worldometer_data['Case Fatality'], 
    name = 'Case Fatality Rate'
))

fig.update_layout(barmode='overlay', 
    title = 'Estimated Infection Fatality Rate vs. Case Fatality Rate',
    xaxis_title_text='Value', # xaxis label
    yaxis_title_text='Count', # yaxis label
)
                  
fig.update_traces(opacity=0.75)

fig.show()

<a id='section-four'></a>
# 4. CONCLUSION

The country-level Covid19 pandemic curves are different around the world. Which shape a curve takes depends on government interventions (i.e., the type and the extent), pandemic preparedness, and healthcare infrastructure. Taking a deep dive into the official Covid19 figures, we discover an issue with the reported confirmed cases and case fatality rates. These numbers depend crucially on the testing intensity. Then, we measure the fraction of tests that returns a positive case (case positivity) and discover that this measure correlates with the case fatality rate. The fraction of population who is aged 65+ is positively correlated with case fatality rate too. We exploit this finding to create Infections Fatality Ratio, which is plausibly closer to the true state of the severity of Covid19.

<a id='section-five'></a>
# 5. REFERENCES

This notebook is partially adapted from code written by [Devakumar KP](https://www.kaggle.com/imdevskp).

Datasets are from [Worldometers](https://www.worldometers.info/) and [COVID-19 Data Repository by the Center for Systems Science and Engineering (CSSE) at Johns Hopkins University](https://github.com/CSSEGISandData/COVID-19). [Devakumar KP](https://www.kaggle.com/imdevskp) has kindly extracted, cleaned, and shared these datasets on Kaggle.